<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW4/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Homework 4

In [1]:
!pip install -qU transformers
!pip install -qU torchtext
!pip install -qU fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3943, done.
remote: Counting objects: 100% (1002/1002), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 3943 (delta 903), reused 859 (delta 850), pack-reused 2941
Receiving objects: 100% (3943/3943), 8.25 MiB | 42.05 MiB/s, done.
Resolving deltas: 100% (2510/2510), done.


In [4]:
import fasttext.util

In [ ]:
!sudo pip install fastText/.
!cd fastText
fasttext.util.download_model('fa', if_exists='ignore')
ft = fasttext.load_model('cc.fa.300.bin')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fastText
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4384288 sha256=f778e46d15aa1278d76a806135be813db3ee0d4ec7927fcf2373ca19229cc2db
  Stored in directory: /tmp/pip-ephem-wheel-cache-njk8quld/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/nlp
%mkdir nlp_hw4
%cd nlp_hw4

##Essential packages

In [ ]:
import pandas as pd
import numpy as np
from numpy import pad
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline
import matplotlib.ticker as ticker
from collections import Counter
from __future__ import unicode_literals, print_function, division
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import seaborn as sns
import fasttext.util


import torch
from torch import nn
import torchtext
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F
from torch import optim

import nltk
from nltk.tokenize import sent_tokenize

from transformers import pipeline, AutoTokenizer

##Load data and analysis


In [ ]:
!unzip archive.zip

df = pd.read_csv('data.csv')
df.head(3)

##Document classification

###Logistic regression

####Preprocess data<br/>implement the cross validation function

####Train and evaluation section

####Test

###Transformer-based model

####Preprocess data(add token, tokenizaiton, padding)

####Train and evaluation section

####Test

##Token classification

###GRU and LSTM

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self,
        num_classes,
        batch_size=10,
        embedding_dim=100,
        hidden_dim=50,
        vocab_size=128):

        super(LSTMClassifier, self).__init__()

        initrange = 0.1

        self.num_labels = num_classes
        n = len(self.num_labels)
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data.uniform_(-initrange, initrange)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)  # !

        print("# !")

        bi_grus = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)
        reverse_gru = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=False)

        self.lstm.weight_ih_l0_reverse = bi_grus.weight_ih_l0_reverse
        self.lstm.weight_hh_l0_reverse = bi_grus.weight_hh_l0_reverse
        self.lstm.bias_ih_l0_reverse = bi_grus.bias_ih_l0_reverse
        self.lstm.bias_hh_l0_reverse = bi_grus.bias_hh_l0_reverse

        bi_output, bi_hidden = bi_grus()
        reverse_output, reverse_hidden = reverse_gru()

        print("# !")

        # self.classifier = nn.Linear(hidden_dim, self.num_labels[0])
        self.classifier = nn.Linear(2 * hidden_dim, self.num_labels[0])  # !


    def repackage_hidden(h):
        """Wraps hidden states in new Tensors, to detach them from their history."""

        if isinstance(h, torch.Tensor):
            return h.detach()
        else:
            return tuple(repackage_hidden(v) for v in h)


    def forward(self, sentence, labels=None):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)  # lstm_out - 2 tensors, _ - hidden layer
        print(lstm_out[:,-1,:])
        tag_space = self.classifier(lstm_out[:,-1,:] + lstm_out[:,-1,:])  # !  # lstm_out[:,-1,:] - 1 tensor
        logits = F.log_softmax(tag_space, dim=1)
        loss = None
        if labels:
            loss = F.cross_entropy(logits.view(-1, self.num_labels[0]), labels[0].view(-1))
        return loss, logits

###Transformer-based model